In [23]:
import numpy as np
import pandas as pd
import yaml
import os
import cv2
import shutil
import torch
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco

METAINFO = {
    "classes": (
        "unlabelled",
        "asphalt/concrete",
        "dirt",
        "mud",
        "water",
        "gravel",
        "other-terrain",
        "tree-trunk",
        "tree-foliage",
        "bush/shrub",
        "fence",
        "other-structure",
        "pole",
        "vehicle",
        "rock",
        "log",
        "other-object",
        "sky",
        "grass",
    ),
    "palette": [
        (0  , 0  , 0  ),
        (230, 25 , 75 ),
        (60 , 180, 75 ),
        (255, 225, 25 ),
        (0  , 130, 200),
        (145, 30 , 180),
        (70 , 240, 240),
        (240, 50 , 230),
        (210, 245, 60 ),
        (250, 190, 190),
        (0  , 128, 128),
        (170, 110, 40 ),
        (255, 250, 200),
        (128, 0  , 0  ),
        (170, 255, 195),
        (128, 128, 0  ),
        (255, 215, 180),
        (0  , 0  , 128),
        (128, 128, 128),
    ],
    "cidx": list(range(19))
}

# Parameters
NUM_CLASSES = 19
id_labels = {i: e for i, e in enumerate(METAINFO['classes'])}

In [24]:
PATH = 'dataset/'
def get_dfs():
    train_df = pd.read_csv(PATH + 'train.csv')
    val_df   = pd.read_csv(PATH + 'val.csv'  )
    test_df  = pd.read_csv(PATH + 'test.csv' )
    return (train_df, val_df, test_df)

train_df, val_df, test_df = get_dfs()

In [24]:
datasets = ['train' , 'val' ]
dfs      = [train_df, val_df]
for i, ds in enumerate(datasets):
    images_destination = f'dataset\\{ds}\\images'
    labels_destination = f'dataset\\{ds}\\labels'
    create_yolo_labels(
        images_destination, labels_destination, dfs[i], NUM_CLASSES,
        # create_path=True
    )

In [4]:
def create_yolo_labels(images_dest, labels_dest, df, n_class, create_path=False):
    if not os.path.exists(labels_dest):
        os.makedirs(labels_dest)
    if not os.path.exists(images_dest):
        os.makedirs(images_dest)

    for image_path, label_path in zip(df['im_path'], df['label_path']):
        image = os.path.basename(image_path)

        # Construct the full path to the destination label_txt
        if create_path:
            image_dest = os.path.join(images_dest, image)
            if not os.path.exists(image_dest):
                shutil.copy(image_path, image_dest)

        # create yolo labels
        label_dest = image.replace('.png', '.txt')
        convert_label_to_yolo(
            label_path, f'{labels_dest}\\{label_dest}', n_class
        )


def convert_label_to_yolo(label_path, out_path, n_class):
    label_img = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)
    height, width = label_img.shape

    with open(out_path, 'w') as label_txt:
        for class_id in range(n_class):
            # Create a binary mask for the current class
            class_mask = cv2.inRange(label_img, class_id, class_id)

            # Find contours for the current class
            contours, _ = cv2.findContours(
                class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )

            for cnt in contours:
                if len(cnt) < 3:
                    continue
                normalized_points = [
                    (point[0][0] / width, point[0][1] / height) for point in cnt
                ]
                label_txt.write(
                    f'{class_id} ' + " ".join(
                        f'{x:.6f} {y:.6f}' for x, y in normalized_points
                    ) + "\n"
                )

In [8]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0)
print(torch.cuda.is_available())

model = YOLO('yolov8s-seg.yaml')
model.model.nc    = NUM_CLASSES
model.model.names = METAINFO['classes']

True


In [9]:
# Train the model
model.train(
    data='dataset.yaml', epochs=50, batch=16, verbose=False,
    # optimizer=AdamW,
    device=0,
    # imgsz=640
)

Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=segment, mode=train, model=yolov8s-seg.yaml, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, str

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 16.3MB/s]
C:\Users\thedu\OneDrive\Desktop\venv\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

AMP: checks passed 


C:\Users\thedu\OneDrive\Desktop\venv\Lib\site-packages\ultralytics\utils\checks.py:651: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):


AMP: checks passed 
AMP: checks passed 


...

      25/50      8.44G      1.814      3.309        1.6      1.633        198        640: 100%|██████████| 379/379 [08:
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

                   all        283      16848      0.364      0.214      0.197      0.102      0.302      0.148       0.14     0.0598
                   all        283      16848      0.364      0.214      0.197      0.102      0.302      0.148       0.14     0.0598
                   all        283      16848      0.364      0.214      0.197      0.102      0.302      0.148       0.14     0.0598



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      26/50      10.5G       1.81      3.309      1.596       1.62       1419        640:  21%|██        | 79/379 [00:5


KeyboardInterrupt: 

In [28]:
np.array(test_df['im_path'].head(5))

array(['WildScenes2d/V-02/image/1623370893-376340233.png', 'WildScenes2d/V-02/image/1623370896-529507864.png', 'WildScenes2d/V-02/image/1623370900-344543609.png', 'WildScenes2d/V-02/image/1623370902-689996803.png', 'WildScenes2d/V-02/image/1623370905-369350905.png'], dtype=object)

In [30]:
results = model(list(test_df['im_path'].head(5)))  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen




0: 480x640 (no detections), 59.6ms
1: 480x640 (no detections), 59.6ms
2: 480x640 (no detections), 59.6ms
3: 480x640 (no detections), 59.6ms
4: 480x640 (no detections), 59.6ms
0: 480x640 (no detections), 59.6ms
1: 480x640 (no detections), 59.6ms
2: 480x640 (no detections), 59.6ms
3: 480x640 (no detections), 59.6ms
4: 480x640 (no detections), 59.6ms
0: 480x640 (no detections), 59.6ms
1: 480x640 (no detections), 59.6ms
2: 480x640 (no detections), 59.6ms
3: 480x640 (no detections), 59.6ms
4: 480x640 (no detections), 59.6ms
Speed: 3.1ms preprocess, 59.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Speed: 3.1ms preprocess, 59.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Speed: 3.1ms preprocess, 59.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


In [17]:
# Visualize results
def visualize(image, label, prediction, palette):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(image)
    axes[0].set_title('Original Image')
    axes[1].imshow(label_to_rgb(label, palette))
    axes[1].set_title('Ground Truth')
    axes[2].imshow(label_to_rgb(prediction, palette))
    axes[2].set_title('Prediction')
    plt.show()

results = model.val(data='your_data.yaml')

# Accessing IoU (assuming results contain IoU)
iou_score = results.metrics['box'].iou

with torch.no_grad():
    for data in test_loader:
        images = data['image'].to(device)
        labels = data['label'].cpu().numpy()
        outputs = model(images).argmax(1).cpu().numpy()
        for i in range(len(images)):
            image = images[i].cpu().numpy().transpose(1, 2, 0)
            label = labels[i]
            prediction = outputs[i]
            visualize(image, label, prediction, METAINFO['palette'])
        break

Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Ultralytics YOLOv8.2.63  Python-3.12.2 torch-2.4.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=segment, mode=train, model=yolov8s-seg.yaml, data=coco8-seg.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1

100%|███████████████████████████████████████████████████████████████████████████████| 439k/439k [00:00<00:00, 18.8MB/s]
Unzipping C:\Users\thedu\OneDrive\Desktop\venv\Studies\9517\Assignment\COMP9517\coco8-seg.zip to C:\Users\thedu\OneDriv

Dataset download success  (1.8s), saved to C:\Users\thedu\OneDrive\Desktop\venv\Studies\9517\Assignment\COMP9517

Dataset download success  (1.8s), saved to C:\Users\thedu\OneDrive\Desktop\venv\Studies\9517\Assignment\COMP9517

Dataset download success  (1.8s), saved to C:\Users\thedu\OneDrive\Desktop\venv\Studies\9517\Assignment\COMP9517

Overriding model.yaml nc=19 with nc=80
Overriding model.yaml nc=19 with nc=80
Overriding model.yaml nc=19 with nc=80

                   from  n    params  module                                       arguments                     

                   from  n    params  module                                       arguments                     

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv            

 21                  -1  1   1969152  ultralytics.nn.modules.block.C2f             [768, 512, 1]                 
 21                  -1  1   1969152  ultralytics.nn.modules.block.C2f             [768, 512, 1]                 
 22        [15, 18, 21]  1   2801504  ultralytics.nn.modules.head.Segment          [80, 32, 128, [128, 256, 512]]
 22        [15, 18, 21]  1   2801504  ultralytics.nn.modules.head.Segment          [80, 32, 128, [128, 256, 512]]
 22        [15, 18, 21]  1   2801504  ultralytics.nn.modules.head.Segment          [80, 32, 128, [128, 256, 512]]
YOLOv8s-seg summary: 261 layers, 11,821,056 parameters, 11,821,040 gradients, 42.9 GFLOPs
YOLOv8s-seg summary: 261 layers, 11,821,056 parameters, 11,821,040 gradients, 42.9 GFLOPs
YOLOv8s-seg summary: 261 layers, 11,821,056 parameters, 11,821,040 gradients, 42.9 GFLOPs



TensorBoard: Start with 'tensorboard --logdir runs\segment\train24', view at http://localhost:6006/
TensorBoard: Start with 'tensorboard --logdir runs\segmen

FileNotFoundError: [34m[1mtrain: [0mError loading data from C:\Users\thedu\OneDrive\Desktop\venv\Studies\9517\Assignment\datasets\coco8-seg\images\train
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [6]:
# def extract_bounding_boxes(mask, class_id):
#     binary_mask = np.where(mask == class_id, 1, 0).astype(np.uint8)
#     contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     bounding_boxes = []
#     for contour in contours:
#         x, y, w, h = cv2.boundingRect(contour)
#         bounding_boxes.append((x, y, w, h))
#     return bounding_boxes

# def normalize_bbox(x, y, w, h, img_width, img_height):
#     x_center = (x + w / 2) / img_width
#     y_center = (y + h / 2) / img_height
#     width = w / img_width
#     height = h / img_height
#     return x_center, y_center, width, height

# def write_yolo_annotation(filename, class_id, bboxes):
#     with open(filename, 'w') as file:
#         for bbox in bboxes:
#             file.write(f'{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n')

# def convert_mask_to_yolo(mask_path, output_path):
#     mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#     img_height, img_width = mask.shape

#     unique_classes = np.unique(mask)
#     for class_id in unique_classes:
#         if class_id == 0:
#             continue  # Skip background
#         bounding_boxes = extract_bounding_boxes(mask, class_id)
#         normalized_boxes = [
#             normalize_bbox(x, y, w, h, img_width, img_height)
#             for x, y, w, h in bounding_boxes
#         ]
#         write_yolo_annotation(output_path, class_id, normalized_boxes)

# def create_yolo_labels(images_destination, labels_destination, df):
#     if not os.path.exists(labels_destination):
#         os.makedirs(labels_destination)

#     for image_path, label_path in zip(df['im_path'], df['label_path']):
#         label_path = label_path.replace('indexLabel', 'label')
#         image = os.path.basename(image_path)

#         # Construct the full path to the destination file
#         image_dest = os.path.join(images_destination, image)
#         shutil.copy(image_path, image_dest)

#         # create yolo labels
#         label = image.replace('.png', '.txt')
#         convert_mask_to_yolo(label_path, f'{labels_destination}\\{label}')

In [ ]:
# def convert_mask_to_yolo(img_path, label_path):
#     yolo_masks = []
#     masks = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

#     for i, mask in enumerate(masks):
#         # Convert the mask to a binary image
#         binary_mask = triangle_threshold(mask) # .astype(np.uint8)

#         # Find the contours of the mask
#         contours, hierarchy = cv2.findContours(
#             binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
#         )

#         # Get the largest contour based on area
#         largest_contour = max(contours, key=cv2.contourArea)

#         # Get the segmentation mask for object 
#         segmentation = largest_contour.flatten().tolist()
#         mask = segmentation

#         # convert mask to numpy array of shape (N,2)
#         mask = np.array(mask).reshape(-1,2)

#         # normalize the pixel coordinates
#         width, height = mask.shape
#         mask_norm = mask / np.array([width, height])

#         # compute the bounding box
#         xmin, ymin = mask_norm.min(axis=0)
#         xmax, ymax = mask_norm.max(axis=0)

#         # concatenate label and mask to obtain YOLO seg format
#         label = str(int(input_label[i].numpy())) + ' '
#         m_norm = ' '.join(map(str, mask_norm.reshape(-1)))
#         yolo_line = label + m_norm +'\n'
#         yolo_masks.append(yolo_line)

#     # os.chdir('Segmented_labels')
#     with open(label_path, 'a') as f:
#         for val in yolo_masks:
#             f.write(val)
#         f.write('\n')